In [ ]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

class HierarchicalClustering:
    def __init__(self, similarity_matrix):
        """
        Initialize the hierarchical clustering with a similarity matrix.

        :param similarity_matrix: A numpy array representing the pairwise similarity graph.
        """
        self.similarity_matrix = similarity_matrix
        self.graph = nx.from_numpy_array(similarity_matrix)
        self.n = similarity_matrix.shape[0]
        self.tree = {}

    def sparsest_cut(self, subset):
        """
        Compute an approximate sparsest cut using spectral methods.

        :param subset: The subset of nodes to divide.
        :return: Two subsets resulting from the sparsest cut.
        """
        subgraph = self.graph.subgraph(subset)
        laplacian = nx.laplacian_matrix(subgraph).toarray().astype(float)

        # Compute the second smallest eigenvector of the Laplacian (Fiedler vector)
        eigenvalues, eigvecs = eigh(laplacian)
        fiedler_vector = eigvecs[:, 1]  # Second smallest eigenvector

        # Partition based on sign of the Fiedler vector
        subset_array = np.array(list(subset))
        left = subset_array[fiedler_vector <= 0]
        right = subset_array[fiedler_vector > 0]
        return set(left), set(right)

    def recursive_clustering(self, subset):
        """
        Recursively split the subset into two parts and build the hierarchy.

        :param subset: The current subset of nodes to split.
        :return: A tree structure for this subset.
        """
        if len(subset) <= 1:
            return frozenset(subset)  # Leaf node as frozenset

        left, right = self.sparsest_cut(subset)
        self.tree[frozenset(subset)] = (frozenset(left), frozenset(right))

        self.recursive_clustering(left)
        self.recursive_clustering(right)

    def build_hierarchy(self):
        """
        Build the hierarchical clustering tree.
        """
        all_nodes = set(range(self.n))
        self.recursive_clustering(all_nodes)

    def compute_cost(self, tree=None):
        """
        Compute the cost of the hierarchical clustering.

        :param tree: The tree structure (defaults to the built tree).
        :return: Total cost of the hierarchy.
        """
        if tree is None:
            tree = self.tree

        def find_leaves(node):
            """
            Recursively find all leaves in the subtree rooted at the given node.
            """
            if len(node) == 1:  # Leaf node
                return set(node)

            left, right = tree[node]
            return find_leaves(left) | find_leaves(right)

        def find_smallest_subtree_size(edge):
            """
            Find the number of leaves in the smallest subtree containing both endpoints of an edge.
            """
            def traverse(node):
                if len(node) == 1:  # Leaf node
                    return node

                left, right = tree[node]

                if edge[0] in left and edge[1] in left:
                    return traverse(left)
                elif edge[0] in right and edge[1] in right:
                    return traverse(right)
                else:
                    return node

            smallest_subtree = traverse(frozenset(range(self.n)))
            return len(find_leaves(smallest_subtree))

        total_cost = 0
        for i, j in self.graph.edges():
            weight = self.graph[i][j]['weight']
            subtree_size = find_smallest_subtree_size((i, j))
            print(f"Edge ({i}, {j}), Weight: {weight}, Subtree Size: {subtree_size}, Contribution: {weight * subtree_size}")
            total_cost += weight * subtree_size

        return total_cost

# Example
if __name__ == "__main__":
    # Example similarity matrix
    similarity_matrix = np.array([
        [0, 1, 0.8, 0, 0.3, 0],
        [1, 0, 0.7, 0, 0, 0.4],
        [0.8, 0.7, 0, 0.5, 0, 0],
        [0, 0, 0.5, 0, 1, 0.9],
        [0.3, 0, 0, 1, 0, 0.8],
        [0, 0.4, 0, 0.9, 0.8, 0]
    ])

    clustering = HierarchicalClustering(similarity_matrix)
    clustering.build_hierarchy()

    print("Tree structure:")
    for parent, children in clustering.tree.items():
        print(f"{set(parent)} -> {set(children[0])}, {set(children[1])}")

    print("Total cost:", clustering.compute_cost())


Tree structure:
{0, 1, 2, 3, 4, 5} -> {3, 4, 5}, {0, 1, 2}
{3, 4, 5} -> {3, 4}, {5}
{3, 4} -> {3}, {4}
{0, 1, 2} -> {0, 1}, {2}
{0, 1} -> {0}, {1}
Edge (0, 1), Weight: 1.0, Subtree Size: 2, Contribution: 2.0
Edge (0, 2), Weight: 0.8, Subtree Size: 3, Contribution: 2.4000000000000004
Edge (0, 4), Weight: 0.3, Subtree Size: 6, Contribution: 1.7999999999999998
Edge (1, 2), Weight: 0.7, Subtree Size: 3, Contribution: 2.0999999999999996
Edge (1, 5), Weight: 0.4, Subtree Size: 6, Contribution: 2.4000000000000004
Edge (2, 3), Weight: 0.5, Subtree Size: 6, Contribution: 3.0
Edge (3, 4), Weight: 1.0, Subtree Size: 2, Contribution: 2.0
Edge (3, 5), Weight: 0.9, Subtree Size: 3, Contribution: 2.7
Edge (4, 5), Weight: 0.8, Subtree Size: 3, Contribution: 2.4000000000000004
Total cost: 20.800000000000004
